In [ ]:
import dysts.flows
from dysts.base import DynSys

from ecodyna.mutitask_models import MultiTaskRNN
from ecodyna.plot import plot_3d_trajectories

In [ ]:
def a(attractor_name: str,):
    attractor: DynSys = getattr(dysts.flows, attractor_name)()

    plot_3d_trajectories()

In [ ]:
data_parameters = {'trajectory_count': 1000, 'trajectory_length': 100}
in_out_parameters = {'n_in': 5, 'n_out': 5}
common_model_parameters = {'model': 'LSTM', 'n_hidden': 32, 'n_layers': 1, **in_out_parameters}
experiment_parameters = {'n_epochs': 5, 'train_part': 0.75, 'n_splits': 2}
dataloader_parameters = {'batch_size': 64, 'num_workers': 8, **in_out_parameters}

models_and_params = [
    (MultiTaskRNN, {'forecast_type': 'n_out'}),
    (MultiTaskRNN, {'forecast_type': 'recurrent'})
]